In [ ]:
pip install git+https://github.com/amsehili/auditok

  Cloning https://github.com/amsehili/auditok to /tmp/pip-req-build-yvzc0zh7
  Running command git clone --filter=blob:none --quiet https://github.com/amsehili/auditok /tmp/pip-req-build-yvzc0zh7
  Resolved https://github.com/amsehili/auditok to commit 5cce0d5cc76d4d2af53baf2be2adf3b929596777
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for auditok: filename=auditok-0.2.0-py3-none-any.whl size=1491678 sha256=11d85b4a67aa882189a21b9906d76a2127745f0af8d9222230269f461391dfad
  Stored in directory: /tmp/pip-ephem-wheel-cache-gymkjx4d/wheels/b1/e9/9e/d0920052c410cd8094d4473de75503ec6f1599d033c19e4872
Successfully built auditok


In [ ]:
pip install pydub

In [ ]:
import auditok

audio_regions = auditok.split(
    "/content/PM Narendra Modi's 78th Independence Day Speech from Red Fort.mp3",
    min_dur=0.2, # minimum duration of a valid audio event in seconds
    max_dur=7,       # maximum duration of an event
    # maximum duration of tolerated continuous silence within an event
    energy_threshold=55 # threshold of detection
)

for i, r in enumerate(audio_regions):
    print("Region {i}: {r.meta.start:.3f}s -- {r.meta.end:.3f}s".format(i=i, r=r))
    filename = r.save("region_{:.0f}-{:.0f}.wav".format(r.meta.start * 1000, r.meta.end * 1000))
    print("region saved as: {}".format(filename))


Region 0: 0.050s -- 1.100s
region saved as: region_50-1100.wav
Region 1: 1.150s -- 1.750s
region saved as: region_1150-1750.wav
Region 2: 4.600s -- 5.750s
region saved as: region_4600-5750.wav
Region 3: 5.900s -- 6.950s
region saved as: region_5900-6950.wav
Region 4: 8.700s -- 10.100s
region saved as: region_8700-10100.wav
Region 5: 10.200s -- 11.300s
region saved as: region_10200-11300.wav
Region 6: 14.250s -- 16.150s
region saved as: region_14250-16150.wav
Region 7: 19.700s -- 21.500s
region saved as: region_19700-21500.wav
Region 8: 25.850s -- 26.550s
region saved as: region_25850-26550.wav
Region 9: 28.200s -- 29.750s
region saved as: region_28200-29750.wav
Region 10: 31.500s -- 33.100s
region saved as: region_31500-33100.wav
Region 11: 34.800s -- 36.300s
region saved as: region_34800-36300.wav
Region 12: 38.150s -- 40.050s
region saved as: region_38150-40050.wav
Region 13: 41.750s -- 44.450s
region saved as: region_41750-44450.wav
Region 14: 46.600s -- 48.900s
region saved as: reg

In [ ]:
import os
import requests
import pandas as pd
from datetime import timedelta
import glob
import re

def ms_to_time(ms):
    seconds = ms / 1000
    td = timedelta(seconds=seconds)
    time_format = str(td)
    if '.' not in time_format:
        time_format += ".000"  # Add milliseconds if missing
    else:
        time_format = time_format[:12]  # Ensure it's in HH:MM:SS.mmm format
    return time_format

def shift_vtt_timestamps(vtt_content, start_ms):
    def shift_time(match):
        original_time = match.group(0)
        time_parts = list(map(float, original_time.replace(':', '.').split('.')))
        total_ms = (time_parts[0] * 3600 + time_parts[1] * 60 + time_parts[2]) * 1000
        new_total_ms = total_ms + start_ms
        return ms_to_time(new_total_ms)

    vtt_updated = re.sub(r'(\d{2}:\d{2}:\d{2}\.\d{3})', shift_time, vtt_content)
    return vtt_updated

data = []

file_list = glob.glob("/content/region*.wav")

for file_path in file_list:
    filename = os.path.basename(file_path)
    start_ms, end_ms = map(int, filename.replace('region_', '').replace('.wav', '').split('-'))

    start_time_vtt = ms_to_time(start_ms)
    end_time_vtt = ms_to_time(end_ms)

    files = {
        'file': open(file_path, 'rb'),
        'language': (None, 'hindi'),
        'vtt': (None, 'true'),
    }

    response = requests.post('https://asr.iitm.ac.in/internal/asr/decode', files=files)
    result = response.json()

    vtt = result.get('vtt', '')

    vtt_shifted = shift_vtt_timestamps(vtt, start_ms)

    data.append([start_time_vtt, end_time_vtt, vtt_shifted])
df = pd.DataFrame(data, columns=['start_time', 'end_time', 'vtt'])

df_sorted = df.sort_values(by='start_time')

df_sorted.to_csv("/content/vtt_output.csv", index=False)

print(df_sorted)


In [ ]:
df_sorted.to_excel("/content/krish_8_seconds.xlsx", index=False)

In [ ]:
import auditok

audio_regions = auditok.split(
    "/content/PM Narendra Modi's 78th Independence Day Speech from Red Fort.mp3",
    min_dur=0.2, # minimum duration of a valid audio event in seconds
    max_dur=7,       # maximum duration of an event
    # maximum duration of tolerated continuous silence within an event
    energy_threshold=20 # threshold of detection
)

for i, r in enumerate(audio_regions):
    print("Region {i}: {r.meta.start:.3f}s -- {r.meta.end:.3f}s".format(i=i, r=r))
    filename = r.save("region_{:.0f}-{:.0f}.wav".format(r.meta.start * 1000, r.meta.end * 1000))
    print("region saved as: {}".format(filename))

In [ ]:
import requests
files = {
        'file': open('/content/region_27900-34900.wav', 'rb'),
        'language': (None, 'hindi'),
        'vtt': (None, 'true'),
    }
response = requests.post('https://asr.iitm.ac.in/internal/asr/decode', files=files)
result = response.json()

In [ ]:
result

{'status': 'success',
 'time_taken': '0.31',
 'transcript': 'वह शुभ घडी है। मन',
 'vtt': 'WEBVTT\n\n00:00:00.000 --> 00:00:05.500\nवह शुभ घडी है।\n\n00:00:06.400 --> 00:00:07.000\nमन'}